In [7]:
import os
from secret_key import open_api_key

os.environ["OPENAI_API_KEY"]=open_api_key

In [8]:
from langchain.llms import OpenAI

In [ ]:
from langchain.llms import 

In [9]:
llm=OpenAI(temperature=0.5)

In [10]:
name=llm("I want to open a new restuarant that primarily serves Indian food. Suggest a fancy name for it.")
name

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


KeyboardInterrupt: 